### 학습 목표

- 초대용량 데이터는 어떻게 처리할까요?


- 데이터를 분할하여 불러오고 처리하는 방법에 대해서 연습해봅니다.


- 불량 검출 문제에 대해 살펴봅니다.


- Class imbalance 문제를 해결하는 방법에 대해 알아봅니다.

## Bosch Production Line Performance

- 공정과정 데이터를 통해 제품 내부 불량 검출하기

- 주어진 공정변수를 이용하여 정상/불량 분류하기

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

train_cat_part = pd.read_csv("../input/bosch-production-line-performance/train_categorical.csv.zip", nrows=1000)
train_numeric_part = pd.read_csv("../input/bosch-production-line-performance/train_numeric.csv.zip", nrows=1000)
train_date_part = pd.read_csv("../input/bosch-production-line-performance/train_date.csv.zip", nrows=1000)

print(train_numeric_part.shape, train_cat_part.shape, train_date_part.shape)


In [ ]:
train_date_part # start_station, end_staion 

### train_date 데이터를 요약해봅시다.
- 각 row마다 측정이 처응된 station 번호와, 마지막으로 측정이 된 station번호를 찾습니다.

In [ ]:
train_date_part

In [ ]:
row = train_date_part.loc[0]
row[row.index.str.contains("_S43_")]# 0.01 == 6분

# 1.station별 column 찾기 : 1157개 -> 52개
# row.index.str.split('_', expand=True)

counts = train_date_part.drop(columns="Id").count()
date_cols = counts.reset_index()["index"].str.split("_", expand=True)
col_idx = date_cols.drop_duplicates(1).index # 1번 열 기준으로 중복 제거 
date_cols = train_date_part.drop(columns="Id").columns[col_idx]
date_cols

In [ ]:
#2. start_station, end_station 찾기
train_date = pd.read_csv("../input/bosch-production-line-performance/train_date.csv.zip", usecols=date_cols, nrows=1000)

## 전체 데이터에서 column 순서대로 scan하면서 처음으로 NaN이 아닌 column의 station이 start_station,
## 마지막까지 scan하면서 마지막으로 NaN이 아니었던 column의 station이 end_station
train_date["start_station"] = -1
train_date["end_station"] = -1

for col in train_date.drop(columns="Id").columns:
    nulls = train_date[col].isnotnull() # L0_S0_D1
    station_name = col.split("_")[1]
    print(station_name)
    display(notnulls)
    
    train_date[col].loc[notnulls & train_date[col].start , "start_station"] = station_name
    
    break


In [ ]:
#train_numeric_part = pd.read_csv("../input/bosch-production-line-performance/train_numeric.csv.zip", usecols=["Id", "Response"])
#train_numeric_part

In [ ]:
#train_numeric_part

In [ ]:
train_numeric_part.Response.value_counts(normalize=True)
sns.countplot(data=train_numeric_part, y="Response") # Target Value Distribution
## Class imbalance problem, 클래스 불균형 문제!

In [ ]:
train_numeric_part.head(10)

### 1. 대용량 데이터 다루기

Technique1. 잘라서 불러오기


Technique2. Reducing Memory Technique (dtype 변경하기)shape

In [ ]:
missing_ratio = pd.Series(index=train_numeric_part.columns, data=np.zeros(len(train_numeric_part.columns))) #970
column_means = pd.Series(index=train_numeric_part.columns, data=np.zeros(len(train_numeric_part.columns))) #970

display(missing_ratio)

# io chunk 사용

#### 대용량 데이터가 메인 메모리에 한번에 적재가 안될때.
1. 사용할 컬럼을 몇개 불러온다 (usecols)
2. 데이터를 쪼개서 임시로 불러와서 계산하고, 다시 불러온다.(chunksize)

In [ ]:
# Technique 1.
length = 0
for chunk in pd.read_csv("../input/bosch-production-line-performance/train_numeric.csv.zip", 
                         
                         chunksize=100000):
    #display(chunk)
    missing_ratio = missing_ratio + chunk.isnull().sum() # 결측치 합
    temp2 = chunk.sum()
    length = length + len(chunk)
    column_means = column_means + temp2 # 누적 평균
    #break

display(missing_ratio)
display(column_means/length)

## 차원의 저주(Curse of Dimensioality)
- 머신러닝 모델이 데이터가 존재하는 공간이 너무 고차원 일 때, feature 가 너무 많을 때,
- 머신러닝 모델이 예측력을 읽어버리는 문제(성능이 저하)
---
- 해결책
> 차원을 줄입니다!(=column 수를 줄인다.)
> 1. feature selection : 분석에 사용할 column을 직접 고른다.
>> 결측치가 많은 column을 버리거나, 도메인 지식으로 필요한 변수를 고르거나, ....
> 2. feature extraction : dimensionality reduction method(e.g. PCA)를 사용한다.

In [ ]:
missing_ratio = missing_ratio / length

In [ ]:
# 결측치가 50%를 넘는 column을 삭제
missing_ratio > 0.5
train_numeric_part.drop(columns=train_numeric_part.columns[missing_ratio > 0.5]) # 일반적


In [ ]:
# 결측비율이 50%를 넘지 않는 column들만 선택

use_cols = train_numeric_part.columns[missing_ratio <= 0.5]

In [ ]:
train_numeric_part2 = pd.read_csv("../input/bosch-production-line-performance/train_numeric.csv.zip", usecols=["Id", "Response"])
train_numeric_part2

In [ ]:
from sklearn.decomposition import PCA
data = pd.read_csv("../input/bosch-production-line-performance/train_numeric.csv.zip",
                        usecols=use_cols)
data = data.fillna(0) # 결측치를 0으로 채움
ids = data.Id
responses = data.Response
data = data.drop(columns=["Id", "Response"])
pca = PCA(n_components=6)
X = pca.fit_transform(data)
pca_df = pd.DataFrame(columns=[f"PC{i}" for i in range(1, 7)],
                     data=X)
display(pca_df)

In [ ]:
from sklearn.decomposition import PCA
for chunk in pd.read_csv("../input/bosch-production-line-performance/train_numeric.csv.zip", 
                         usecols=use_cols, chunksize=10000):
    
    chunk = chunk.fillna(0) # 0 fill
    #display(chunk)
    temp = chunk.drop(columns=["Id", "Response"])
    #machine learning using PCA
    pca = PCA(n_components=15)
    X = pca.fit_transform(temp)
    pca_df = pd.DataFrame(columns=[f"PC{i}" for i in range(1,16) ], data=X)
    display(pca_df)
    break

In [ ]:
train_numeric_part2.info()

### dtype
- np.int8, np.int16, np.int32, np.int64 : 정수 하나를 몇 bit로 표현할 것인가?
> int8 : -2^7 ~ 2^your 7-1 --> -128 ~ 127 --> 0000000000 ~ 11111111
- np.float32 / np.float64 : 실수 하나를 몇 bit로 표현할 것인가?
> 실수 표현은 항상 근처를 쓰기 때문에, 얼마나 자세하게 표현할지의 차이!

> 필연적으로 발생하는 수치적 차이를 "numerical error"라고 부릅니다.

In [ ]:
# Technique 2. dtype 변경
pca_df.info()

In [ ]:
pca_df.astype(np.float32).info()

### 2. 불량 검출 문제 다루기 (Class Imbalance problem)

Technique 1. Sampling Method (Under / Over)
> binary classification (이진 분류) : 정상 혹은 불량 로 예측하는 문제!

> 일반적으로 불량에 해당하는 케이스가 매우 매우 적습니다. (minority class)

> 일반적으로 정산에 해당하는 케이스가 아주아주 많습니다. (majority class)

> 머신러닝 분류 문제에서는 클래스의 비율이 비슷할 수록 학습이 잘됩니다.

>> minority -> majority (oversampling) 

>> majority -> minority (undersampling)

---
Technique 2. Change to Outlier Detection problem

- 주류(또는 대다수) : majority

- 주류가 아닌 데이터를 outlier(이상치)라고 판단하는 방법.

- IQR, IsolationForest (ML) , DBCAN (ML)


(Avanced) Technique 3. Semi-supervised Learing (DL), MakinaRocks

In [ ]:
# Technique 1.
# X : feature vector (독립변수), 학습 데이터(문제)
# y : target value (종속변수), 목표값 (정답)
X = pca_df
y = responses
print(X.shape, y.shape)

## 1-1. Undersampling - y에서 1인 개수로 0인 데이터를 줄이는 것.
normal = X[y==0]
failure = X[y==1]
print(normal.shape, failure.shape) # 1176 --> 6879

normal_under = normal.sample(n=failure.shape[0], random_state=0xC0FFEE)
print(normal_under.shape, failure.shape)

In [ ]:
# 1-2 Oversampling - yr가 1인 데이터를 y가 0인 데이터의 개수로 늘리는 것.
from imblearn.over_sampling import SMOTE

sm = SMOTE()
X_oversampled, y_oversampled = sm.fit_resample(X,y)
print(X_oversampled.shape, y_oversampled.shape)


In [ ]:
# 1-3 . 

### 정상데이터를 500,000개로 undersampling 한뒤, oversamling을 수행하여
### 데이터를 1,000,000객 되도록 만들어주세요!
normal_hybrid = normal.sample(n=500000, random_state=42)
print(normal_hybrid.shape)
#X_hybrid, y_hybrid = 

In [ ]:
y[normal_hybrid.index].shape

In [ ]:
# 정상 불량을 모두 넣어야 하는데
sm = SMOTE()
X_hybrid, y_hybrid = sm.fit_resample(normal_hybrid,y[normal_hybrid.index])
print(X_hybrid.shape, y_hybrid.shape)

In [ ]:
X_oversampled

In [ ]:
normal.shape[0]*2

In [ ]:
## 1-3. Hybrid Approach : Undersampling + Oversampling

### 정상 데이터를 500,000개로 undersampling한 뒤, 불량 데이터를 oversampling을 수행하여
### 데이터를 1,000,000개가 되도록 만들어주세요!!

X_under = normal.sample(n=500000).reset_index(drop=True)
y_under = np.zeros(len(X_under))
X_total = pd.concat([X_under, failure]).reset_index(drop=True)
y_total = np.hstack([y_under, np.ones(len(failure))]).astype(int)


# SMOTE에 정상과 불량 데이터가 합쳐진 전체 데이터를 넣으면 알아서,
# majority class 크기에 맞게 oversampling이 됩니다.
X_hybrid, y_hybrid = sm.fit_resample(X_total, y_total)
print(X_hybrid.shape, y_hybrid.shape)

In [ ]:
# Technique 2.
sns.boxplot(train_numeric_part.L0_S0_F0)

In [ ]:
def IQR_outlier(data, column):
    temp = data[column].dropna() # 결측치가 없어야 함.
    
    q25 = np.percentile(temp, 25) 
    q75 = np.percentile(temp, 75)
    
    IQR = q75 - q25 # Inter-Quartile Range
    lower_bound = q25 - IQR * 1.5
    upper_bound = q75 + IQR * 1.5
    
    outlier = temp.loc[(temp < lower_bound) | (temp > upper_bound)]
    return outlier
        

In [ ]:
IQR_outlier(train_numeric_part, "L0_S0_F0")